In [ ]:
!pip install opendp

In [ ]:
import numpy as np
import opendp
from opendp.metrics import l1_distance, l2_distance
from opendp.domains import vector_domain, atom_domain
from opendp.measurements import make_base_discrete_laplace, \
                                make_base_discrete_gaussian, \
                                make_base_gaussian, \
                                make_base_laplace, \
                                make_randomized_response_bool, \
                                make_randomized_response

class LocalDifferentialPrivacy:
    def __init__(self, scale, probability):
      self.scale = scale # for laplace noise
      self.probability = probability # for randomized response

    def define_input_space(self, input_space_type='vector', variable_type='float'):
      if input_space_type == 'scalar':
        input_space = atom_domain(T=variable_type), l1_distance(T=variable_type)
      elif input_space_type == 'vector':
        input_space = vector_domain(atom_domain(T=variable_type)), l1_distance(T=variable_type)
      else:
          raise ValueError("Unsupported input type")
      return input_space

    def add_laplace_noise(self, input_space, value, variable_type='float'):
      if variable_type == 'int':
        base_discrete_lap = make_base_discrete_laplace(*input_space, scale=self.scale)
        noisy_value = base_discrete_lap(value)
      elif variable_type == 'float':
        base_lap = make_base_laplace(*input_space, scale=self.scale)
        noisy_value = base_lap(value)
      else:
          raise ValueError("Unsupported variable type")
      return noisy_value

    def randomized_response(self, true_value, variable_type='bool'):
      if variable_type == 'bool':
        rr_measure = make_randomized_response_bool(prob=self.probability)
      elif variable_type == 'categorical':
        rr_measure = make_randomized_response(list(set(true_value)), prob=self.probability)
      else:
          raise ValueError("Unsupported variable type")
      noisy_value = [rr_measure(value) for value in true_value]
      return noisy_value

In [ ]:
# test
privacy = LocalDifferentialPrivacy(1.0, 0.75)

# Add Laplace noise
sensitivity = 10
true_value_float = [8.9, 3.4, 6.2, 10.0, 0.0, 0.3]
true_value_int = [3, 5, 7, 1, 0, 9, 9]
true_value_bool = [True, False, False, True, False]
true_value_category = ['C','A','E','B','B','C','A','D','E','E','E']

# Add Laplace noise
input_space = privacy.define_input_space(input_space_type = 'vector', variable_type = 'float')
noisy_value = privacy.add_laplace_noise(input_space, true_value_float)
print("Laplace noisy value:", noisy_value)

# Add Laplace noise discerete
input_space = privacy.define_input_space(input_space_type = 'vector', variable_type = 'int')
noisy_value = privacy.add_laplace_noise(input_space, true_value_int, variable_type = 'int')
print("Laplace discrete noisy value:", noisy_value)

# Randomized Response boolean
noisy_value = privacy.randomized_response(true_value_bool)
print("Randomized Response noisy value:", noisy_value)

# Randomized Response categorical
noisy_value = privacy.randomized_response(true_value_category, variable_type = 'categorical')
print("Randomized Response noisy value:", noisy_value)